# Create a melt rate dataset

Melt rates are based on Paolo et al., 2023, representing 1992 to 2017 averages.
Uncertainties are based on Adusumilli error estimates as Paolo estimates appear too small.

Output:
- 2D 8km melt rates, and error NetCDF file
- Melt rates and errors aggregated in IMBIE basins


In [1]:
import xarray as xr
import os
import glob
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt

# Load data 

In [2]:
# basins for comparison

data_path = '/media/NAS2/ISMIP7/'

# basin file to optimise deltaT
basins = xr.load_dataset(data_path+"imbie2_basins_for_ISMIP7/basinNumbers_8km.nc")
basins = basins.rename({'basinNumber':'basins'})

# present-day melting, these datasets have been remapped to the 8km grid using nearest neighbor interpolation 
# and the pism-ais toolbox avaialble through github: https://github.com/pism/pism-ais.git
melt_paolo = xr.load_dataset(data_path+"Melt_Rate_Observations/paolo_initmip8km.nc")
melt_adusumilli = xr.load_dataset(data_path+"Melt_Rate_Observations/adusumilli_initmip8km.nc")


ice_density_adusumilli = 917. # kg/m3 see caption of Fig 1 in Adusumilli et al., 2020
ice_density_paolo = 917. # kg/m3 see Paolo et al., 2023

# Create combined dataset - melt paolo, error as max from paolo and adusumilli

In [3]:
melt_data = melt_paolo.copy(deep=True)

In [4]:
melt_data['melt_mean'] = melt_data['melt_mean']*-1*ice_density_adusumilli
melt_data['melt_mean_err'] = np.maximum(melt_adusumilli['w_b_uncert']*ice_density_adusumilli, 
                                        melt_paolo['melt_mean_err']*ice_density_paolo )

melt_data['melt_mean'].attrs['units'] = 'kg/m2/a'
melt_data['melt_mean_err'].attrs['units'] = 'kg/m2/a' 

In [5]:
# Create NetCDF dataset and save
melt_data.to_netcdf(data_path+'/share_with_modellers/meltmip/melt_paolo_err_adusumilli_ismip8km.nc')

# Get data for evaluation of basins from Table

In [10]:
!pip install openpyxl
!pip list | grep openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]━━ 1/2 [openpyxl]
openpyxl                       3.1.5


In [35]:
# Load data 
# This file can be downloaded from Paolo et al., 2023
ice_density_paolo = 917. # see Paolo et al 2023

xl = pd.ExcelFile(data_path+'Melt_Rate_Observations/Paolo23/tc-17-3409-2023-t03.xlsx')
df = xl.parse("Worksheet 1",header=0, usecols=[0,1,3])
df = df.drop(index=0).reset_index(drop=True)

ice_shelf_name = df['Ice shelf']
area = df['Area']# km2
bmr_with_uncert = df['Basal melt rate'] # m/a 

In [40]:
bmr = np.zeros_like(bmr_with_uncert)
bmr_err = np.zeros_like(bmr_with_uncert)
areaf = np.zeros_like(bmr_with_uncert)

for i in range(len(df.index)):
    #print(bmr_with_uncert[i].split())
    bmr[i] = float(bmr_with_uncert[i].replace("−", "-").split()[0])
    bmr_err[i] = float(bmr_with_uncert[i].split()[2])
    tmp = str(area[i])
    #print(tmp)
    areaf[i] = float(tmp.replace(' ', ''))
    #print(areaf[i])

df['Melt rate'] = bmr*ice_density_paolo # convert to kg/m2/a
df['Melt rate error'] = bmr_err*ice_density_paolo   # convert to kg/m2/a
df['Area'] = areaf


In [41]:
# Since there are no lat/lon values, we match to Rignot data

xl = pd.ExcelFile(data_path+'Melt_Rate_Observations/Rignot12/Rignot12_TableS1_formatted.xlsx')
dfr = xl.parse("Sheet1")
ice_shelf_nameR = dfr['Ice Shelf Name']
latR = dfr['Lat (o)']
lonR = dfr['Lon (o)']



In [42]:
lats = np.zeros_like(bmr)
lons = np.zeros_like(bmr)

for i in range(len(df.index)):
    name = df['Ice shelf'][i]
    #print(name)
    idx = (ice_shelf_nameR==name).idxmax()
    #print(idx, ice_shelf_nameR[idx])
    lats[i] = latR[idx]
    lons[i] = lonR[idx]

df['lat'] = lats
df['lon'] = lons


In [46]:

bmrGt = df['Melt rate']*(df['Area']*1000**2)/1e12 # kg/m2/a * m2
bmrGt_err = df['Melt rate error']*(df['Area']*1000**2)/1e12 # kg/m2/a * m2


df['BMR'] = bmrGt
df['BMR err'] = bmrGt_err


In [47]:
# Manually fill gaps 
df.loc[14,'lat']  = df.loc[13,'lat'] 


df.loc[df["Ice shelf"] == 'Thwaites Calved', 'lat'] = df.loc[df["Ice shelf"] == 'Thwaites','lat'].values 
df.loc[df["Ice shelf"] == 'Thwaites Calved', 'lon'] = df.loc[df["Ice shelf"] == 'Thwaites','lon'].values 

df.loc[df["Ice shelf"] == 'Ross East', 'lat'] = df.loc[df["Ice shelf"] == 'Ross West','lat'].values 
df.loc[df["Ice shelf"] == 'Ross East', 'lon'] = df.loc[df["Ice shelf"] == 'Ross West','lon'].values 

df.loc[df["Ice shelf"] == 'Conger Glenzer', 'lat'] = dfr.loc[dfr["Ice Shelf Name"] == 'Conger/Glenzer','Lat (o)'].values 
df.loc[df["Ice shelf"] == 'Conger Glenzer', 'lon'] = dfr.loc[dfr["Ice Shelf Name"] == 'Conger/Glenzer','Lon (o)'].values 

df.loc[df["Ice shelf"] == 'Baudouin', 'lat'] = dfr.loc[dfr["Ice Shelf Name"] == 'Baudouin [9]','Lat (o)'].values 
df.loc[df["Ice shelf"] == 'Baudouin', 'lon'] = dfr.loc[dfr["Ice Shelf Name"] == 'Baudouin [9]','Lon (o)'].values 

df.loc[df["Ice shelf"] == 'Atka', 'lat'] = dfr.loc[dfr["Ice Shelf Name"] == 'Atka [8]','Lat (o)'].values 
df.loc[df["Ice shelf"] == 'Atka', 'lon'] = dfr.loc[dfr["Ice Shelf Name"] == 'Atka [8]','Lon (o)'].values 

df.loc[df["Ice shelf"] == 'Ekström', 'lat'] = dfr.loc[dfr["Ice Shelf Name"] == 'Ekstrom','Lat (o)'].values 
df.loc[df["Ice shelf"] == 'Ekström', 'lon'] = dfr.loc[dfr["Ice Shelf Name"] == 'Ekstrom','Lon (o)'].values 

df.loc[df["Ice shelf"] == 'Brunt Stancomb', 'lat'] = dfr.loc[dfr["Ice Shelf Name"] == 'Brunt/Stancomb','Lat (o)'].values 
df.loc[df["Ice shelf"] == 'Brunt Stancomb', 'lon'] = dfr.loc[dfr["Ice Shelf Name"] == 'Brunt/Stancomb','Lon (o)'].values 



df

,Ice shelf,Area,Basal melt rate,Melt rate,Melt rate error,lat,lon,BMR,BMR err
0,Ronne,327407.1,0.14 ± 0.20,128.38,183.4,-78.4469,-63.50714,42.032523,60.046462
1,Larsen E,1167.3,0.94 ± 0.51,861.98,467.67,-73.4015,-61.0849,1.006189,0.545911
2,Larsen D,22594.3,0.69 ± 0.20,632.73,183.4,-70.7032,-61.724,14.296091,4.143795
3,Larsen C,47443.5,0.54 ± 0.20,495.18,183.4,-67.4601,-62.72942,23.493072,8.701138
4,Larsen B,2151.1,0.06 ± 0.43,55.02,394.31,-65.547,-61.41953,0.118354,0.8482
5,George VI,23259.9,2.85 ± 0.24,2613.45,220.08,-72.346,-69.61707,60.788586,5.119039
6,Wilkins,12906.7,0.95 ± 0.28,871.15,256.76,-70.4869,-72.07288,11.243672,3.313924
7,Bach,4547.9,1.08 ± 0.29,990.36,265.93,-72.0424,-72.1,4.504058,1.209423
8,Stange,7930.0,2.86 ± 0.27,2622.62,247.59,-73.218,-76.67237,20.797377,1.963389
9,Venable,3155.0,4.04 ± 0.37,3704.68,339.29,-73.0893,-87.18664,11.688265,1.07046


In [48]:
# This function finds for a given lat and lon the basin id corresponding to these values

def basinNumberOfNearestNeighbor2(llon,llat):
        i_min = (np.sqrt((lat -llat)**2) + np.sqrt((lon -llon)**2)).argmin(dim=['x','y'])['x'].values
        j_min = (np.sqrt((lat -llat)**2) + np.sqrt((lon -llon)**2)).argmin(dim=['x','y'])['y'].values

        return basins['basins'][j_min,i_min].values

lon=melt_data['lon'] # does not matter which basins here, we just need lat/lon
lat=melt_data['lat']

lonP = df['lon']
latP = df['lat']


compute_basinID_for_shelves = 0

length = len(ice_shelf_name)
if compute_basinID_for_shelves == 1:
        #Load basins data, here: 5km adjusted Zwally basins 

        
        basin_shelf = np.zeros(length) + 100
        for i_shelf in range(length):
                print (i_shelf, ice_shelf_name[i_shelf])
                try:
                    basin_shelf[i_shelf] = basinNumberOfNearestNeighbor2(lonP[i_shelf], latP[i_shelf])
                    print (basin_shelf[i_shelf])
                except:
                    basin_shelf[i_shelf] = 100
                    print ('No basin, setting to nan')
        df['basin'] = basin_shelf
                    
        np.save(data_path+'Melt_Rate_Observations/Paolo23/BasinIds_ofShelves_imbie2', basin_shelf)

        # create a text file with ice_shelf names and basin numbers:
        file = open(data_path+'Melt_Rate_Observations/Paolo23/ShelvesAndBasins_imbie2.txt', 'w')
        for i in range(1,length):
                file.write(str(ice_shelf_name[i])+' '+str(basin_shelf[i])+'\n')
        file.close()
else:
        basin_shelf = np.load(data_path+'Melt_Rate_Observations/Paolo23/BasinIds_ofShelves_imbie2.npy')

In [49]:
PaoloData = pd.DataFrame(index=range(0,basins['basins'].max().values+1),columns =[ 'BMR (Gt/yr)', 'Area (km^2)', 
                                                      'BMR uncert (Gt/yr)', 'Average BMR (kg/m2/a)', 
                                                      'Average BMR uncert (kg/m2/a)' ]).fillna(0.0)

counter = np.zeros(int(basins['basins'].max())+1)

for i_shelf in range(1,length):
        basin_id = basin_shelf[i_shelf]
        if basin_id < 100: # zero only for summed up values in table.. 
                print( basin_id, ice_shelf_name[i_shelf])
                PaoloData.loc[basin_id,'BMR (Gt/yr)']  = PaoloData.loc[basin_id,'BMR (Gt/yr)'] + df.loc[i_shelf,'BMR'] #Gt/yr
                PaoloData.loc[basin_id,'Area (km^2)']  = PaoloData.loc[basin_id,'Area (km^2)'] +  df.loc[i_shelf, 'Area'] # km^2
                PaoloData.loc[basin_id,'BMR uncert (Gt/yr)']  = PaoloData.loc[basin_id,'BMR uncert (Gt/yr)'] + df.loc[i_shelf,'BMR err']**2 # Gt/yr
                counter[int(basin_id)] = counter[int(basin_id)] + 1 

for basin_id in range(int(basins['basins'].max())+1):
    try:
        PaoloData.loc[basin_id,'BMR uncert (Gt/yr)'] = np.sqrt(PaoloData.loc[basin_id,'BMR uncert (Gt/yr)'])/counter[int(basin_id)]
    except:
        print('ISSUE', basin_id)
# ice equivalent, m/a
PaoloData['Average BMR (kg/m2/a)'] = PaoloData['BMR (Gt/yr)']*1e12/(PaoloData['Area (km^2)']*1e6)#/ice_density 
PaoloData['Average BMR uncert (kg/m2/a)'] = PaoloData['BMR uncert (Gt/yr)']*1e12/(PaoloData['Area (km^2)']*1e6)#/ice_density

PaoloData = PaoloData.replace(0.0,np.nan)

PaoloData

13.0 Larsen E
13.0 Larsen D
12.0 Larsen C
12.0 Larsen B
11.0 George VI
11.0 Wilkins
11.0 Bach
11.0 Stange
10.0 Venable
10.0 Abbot
9.0 Cosgrove
9.0 Pine Island
9.0 Thwaites
9.0 Thwaites Calved
9.0 Crosson
9.0 Dotson
8.0 Getz
8.0 Nickerson
7.0 Sulzberger
7.0 Ross West
7.0 Ross East
6.0 Drygalski
6.0 Nansen
6.0 Mariner
6.0 Rennick
5.0 Cook
5.0 Ninnis
5.0 Mertz
5.0 Dibble
4.0 Holmes
4.0 Moscow University
4.0 Totten
3.0 Conger Glenzer
3.0 Tracy/Tremenchus
3.0 Shackleton
3.0 West
3.0 Publications
2.0 Amery
1.0 Prince Harald
0.0 Baudouin
0.0 Borchgrevink
0.0 Lazarev
0.0 Nivl
0.0 Vigrid
0.0 Fimbul
0.0 Jelbart
15.0 Atka
15.0 Ekström
15.0 Quar
15.0 Riiser-Larsen
15.0 Brunt Stancomb
14.0 Filchner


,BMR (Gt/yr),Area (km^2),BMR uncert (Gt/yr),Average BMR (kg/m2/a),Average BMR uncert (kg/m2/a)
0,39.850576,124527.1,1.119366,320.015291,8.988933
1,3.301475,5455.0,1.50067,605.22,275.1
2,22.30045,60797.3,13.38027,366.8,220.08
3,52.685536,49115.8,1.532418,1072.679998,31.200104
4,85.776519,14503.0,1.589851,5914.398351,109.622202
5,20.795532,12576.2,0.591787,1653.562411,47.056075
6,11.458845,10172.4,0.440695,1126.46424,43.3226
7,40.053079,492397.9,12.043937,81.342912,24.459765
8,124.962888,39118.8,5.157949,3194.44583,131.853447
9,258.871849,23007.0,5.435619,11251.873307,236.25934


In [50]:
PaoloData.to_csv(data_path+'Melt_Rate_Observations/Paolo23/PaoloData_imbie2.csv')

In [51]:
pd.read_csv(data_path+'Melt_Rate_Observations/Paolo23/PaoloData_imbie2.csv', index_col=0)

,BMR (Gt/yr),Area (km^2),BMR uncert (Gt/yr),Average BMR (kg/m2/a),Average BMR uncert (kg/m2/a)
0,39.850576,124527.1,1.119366,320.015291,8.988933
1,3.301475,5455.0,1.500670,605.220000,275.100000
2,22.300450,60797.3,13.380270,366.800000,220.080000
3,52.685536,49115.8,1.532418,1072.679998,31.200104
4,85.776519,14503.0,1.589851,5914.398351,109.622202
5,20.795532,12576.2,0.591787,1653.562411,47.056075
6,11.458845,10172.4,0.440695,1126.464240,43.322600
7,40.053079,492397.9,12.043937,81.342912,24.459765
8,124.962888,39118.8,5.157949,3194.445830,131.853447
9,258.871849,23007.0,5.435619,11251.873307,236.259340


# Create combined estimate with Paolo melt and Adusumilli uncertainty

In [52]:
#dfR = pd.read_csv('Rignot12/RignotData_imbie2.csv', index_col=0)
AdusumilliData = pd.read_csv(data_path+'Melt_Rate_Observations/Adusumilli20/AdusumilliData_imbie2.csv', index_col=0)

In [53]:
PaoloData['BMR uncert (Gt/yr)'] < AdusumilliData['BMR uncert (Gt/yr)']

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
Name: BMR uncert (Gt/yr), dtype: bool

In [54]:
MeltData = PaoloData.copy(deep=True)

In [55]:
MeltData['BMR uncert (Gt/yr)'] = AdusumilliData['BMR uncert (Gt/yr)']

In [56]:
MeltData['Average BMR uncert (kg/m2/a)'] = AdusumilliData['Average BMR uncert (m/a)']*ice_density_adusumilli


In [57]:
MeltData.to_csv(data_path+'Melt_Rate_Observations/Paolo23/Melt_Paolo_Err_Adusumilli_imbie2.csv')